In [33]:
import pandas as pd
import numpy as np
import re

In [36]:
file_path = "ufo_data.csv"
df = pd.read_csv(file_path)

df_dropped_nas = df.dropna(subset=["shape", "duration", "occurred_date_time"])


# cleaning function takes in dataframe, column, and patterns to filter out worthless and/or inconsistent values
# function also standardizes the date and filters out the time in "occurred_date_time" column
def clean_dataframe(df, column_name, patterns):
    for pattern in patterns:
        df = df[~df[column_name].str.contains(pattern, regex=True, na=False)]

    df[column_name] = pd.to_datetime(df[column_name], errors="coerce")

    df = df.dropna(subset=[column_name])

    df[column_name] = df[column_name].dt.strftime('%m/%d/%y')
    
    return df

column_name = "occurred_date_time"
patterns = [r'\?', r'unknown', r'&', r'ongoing']

df_cleaned_dates = clean_dataframe(df_dropped_nas, column_name, patterns)

def clean_duration(df, column_name, filter_values):
    pattern = '|'.join(filter_values)
    df_cleaned = df[~df[column_name].str.contains(pattern, case=False, na=False)]

    def convert_to_seconds(duration):
    
        duration = duration.lower()

        match = re.match(r'(\d+)\s*(seconds|minutes|hours?)', duration)

        if match:
            value = int(match.group(1))
            unit = match.group(2)

            if 'second' in unit:
                return value
            elif 'minute' in unit:
                return value * 60
            elif 'hour' in unit:
                return value * 3600
        return None
        
    df_cleaned[column_name] = df_cleaned[column_name].apply(convert_to_seconds)
    df_cleaned = df_cleaned.dropna(subset=[column_name])
    return df_cleaned

filter_values = ["months", "years", "constant", "few", "ongoing"]

df_cleaned_duration = clean_duration(df_cleaned_dates, "duration", filter_values)

df_cleaned_duration.dropna()

C:\Users\kylea\AppData\Local\Temp\ipykernel_20452\2179751516.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[column_name] = pd.to_datetime(df[column_name], errors="coerce")
C:\Users\kylea\AppData\Local\Temp\ipykernel_20452\2179751516.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[column_name] = df_cleaned[column_name].apply(convert_to_seconds)


,id,occurred_date_time,city,state,shape,duration,summary,details_url,posted_date
0,147518,07/12/19,Redding,CA,Cylinder,300.0,"North Redding, CA. We saw a bright flashing w...",http://www.nuforc.org/webreports/147/S147518.html,7/12/19
1,147514,07/12/19,Charlotte,NC,Other,1200.0,Whooshing sound with 1-red and 3-white lights ...,http://www.nuforc.org/webreports/147/S147514.html,7/12/19
2,147511,07/11/19,Cedar Creek,TX,Other,20.0,"Sitting on my front porch a few minutes ago, w...",http://www.nuforc.org/webreports/147/S147511.html,7/12/19
3,147512,07/11/19,San Bernardino,CA,Triangle,120.0,On the freeway on my way home from a distance....,http://www.nuforc.org/webreports/147/S147512.html,7/12/19
4,147509,07/11/19,Bowling Green,KY,Other,2700.0,Me and my girlfriend watched as we first seen ...,http://www.nuforc.org/webreports/147/S147509.html,7/12/19
...,...,...,...,...,...,...,...,...,...
120692,26328,06/08/24,East Peoria,IL,Unknown,300.0,Fast streaking object in the sky with high-pit...,http://www.nuforc.org/webreports/026/S26328.html,12/23/02
120697,52343,06/08/24,Bristol,PA,Disk,15.0,"It was so short, it had to be real. It was unl...",http://www.nuforc.org/webreports/052/S52343.html,10/30/06
120700,21385,06/08/24,Indiana,PA,Oval,120.0,Verifiable sighting,http://www.nuforc.org/webreports/021/S21385.html,2/4/13
120703,43884,06/08/24,Interstate-5,CA,Other,180.0,Fast Moving Spacecraft diving sideways then ri...,http://www.nuforc.org/webreports/043/S43884.html,5/11/05
